In [1]:
import pandas as pd
import numpy as np
import pylab as pl
import os 
import sys
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import scipy.stats as st
import statsmodels.formula.api as smf

%matplotlib inline


# Download Data

In [9]:
def DownloadDataMonth(URL):
    """ Need to fill in"""
    for yr in range(2015,2018):
        # Define year
        year = str(yr)

        for mnth in range(1,13):
            # Define month
            month = ('{:02d}'.format(mnth))

            # Create url and filenames
            url = str(URL) + year + "-" + month + ".csv"
            filename = str(url).split('/')[-1]

            # Yellow Taxis
            os.system("wget " + url)
            os.system("mv " + filename +  " Data")




# Format Taxi Data Sets for Merge

### Taxi Zone Data

In [2]:
# Read in taxi zones
taxizones = gpd.read_file("Data/Taxi_Zone_Shapefile/taxi_zones.shp")

In [3]:
# Convert CRS to functional one
taxizones = taxizones.to_crs(epsg=2263)

In [4]:
# Drop redundant columns
taxizones.drop(['OBJECTID','Shape_Leng','Shape_Area'],axis=1,inplace=True)
taxizones.head()

,zone,LocationID,borough,geometry
0,Newark Airport,1,EWR,"POLYGON ((933100.9183527121 192536.0857092953,..."
1,Jamaica Bay,2,Queens,(POLYGON ((1033269.243591295 172126.0078245941...
2,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.769506665 256767.6975524619,..."
3,Alphabet City,4,Manhattan,"POLYGON ((992073.4667968614 203714.0760008526,..."
4,Arden Heights,5,Staten Island,"POLYGON ((935843.3104932597 144283.3358627402,..."


### Yellow Taxis

In [5]:
###############################################################################
# Yellow Taxi Processing Data Function
def ProcessYTData():
    """ Need to fill in"""
    
    
    # Create Blank Dataframes
    df_Loc = pd.DataFrame(columns=['tpep_pickup_datetime',
                                   'pickup_longitude',
                                   'pickup_latitude'])

    df_ID = pd.DataFrame(columns=['tpep_pickup_datetime',
                                  "PULocationID"])

##############################################################################

    for yr in range(2015,2018):
        # Define year
        year = str(yr)

        for mnth in range(1,13):
            # Define month
            month = ('{:02d}'.format(mnth))
            
            # Download Data and format
            temp = pd.read_csv("Data/yellow_tripdata_" + 
                               year + "-" + month + ".csv")
            
            if(sum(temp.ix[:,-2:].isnull().sum()/len(temp)) == 2.0):
                #fix indexing problem
                columns = temp.columns
                temp = temp.reset_index()
                temp = temp.drop(columns[-2:], axis=1)
                temp.columns = columns

            #CHECK IF LON LAT IS INCLUDED
            if "pickup_longitude" in temp.columns:
                temp = temp[['tpep_pickup_datetime',
                             'pickup_longitude',
                             'pickup_latitude']]

                # Append to larger lat/lon dataframe
                df_Loc = pd.concat([df_Loc, temp]).reset_index(drop=True)

            else:
                temp = temp[['tpep_pickup_datetime',
                                 "PULocationID"]]
                df_ID = pd.concat([df_ID, temp]).reset_index(drop=True)




            # Provide Status Update
            print("Completed " + str(year) + "-" + str(month))
            
###############################################################################            

    # Change Name of columns
    if len(df_Loc) > 0:
        df_Loc.rename(columns={'tpep_pickup_datetime': 'pickup_date'},inplace=True)
        df_Loc['Base'] = 'Yellow'


        # Filter out invalid data points (outside of NYC)
        # Drop Coords out of range 
        #West -74.257159 East -73.699215
        #North 40.915568 South 40.495992
        df_Loc = df_Loc[~((df_Loc.pickup_longitude < -74.5) | (df_Loc.pickup_longitude > -73))]
        df_Loc = df_Loc[~((df_Loc.pickup_latitude < 40) | (df_Loc.pickup_latitude > 41))]

        # Convert Lat Long into point geometry
        crs = {'init':'epsg:4326'}
        geometry = [Point(xy) for xy in zip(df_Loc.pickup_longitude, 
                                            df_Loc.pickup_latitude)]
        df_Loc = GeoDataFrame(df_Loc, crs=crs, geometry=geometry)

        # COnvert Cooords for Spatial join
        df_Loc =  df_Loc.to_crs(epsg=2263)
        print("Coordinates Converted")

        # Conduct Spatial Join to identify tazi zone of pickup
        print("spatial join starting. This may take a while")

        df_Loc = gpd.sjoin(df_Loc, taxizones,op='within')
        print("spatial join complete")

        # Drop Columns
        df_Loc = df_Loc[['pickup_date','LocationID','zone','borough','Base']]

        # Create a date column
        df_Loc.pickup_date = pd.to_datetime(df_Loc.pickup_date)
        df_Loc['date'] = df_Loc.pickup_date.dt.date
        df_Loc['hour'] = df_Loc.pickup_date.dt.hour

    # Merge Location IDs to get Borough and anme of zone
    if len(df_ID) > 0:
        df_ID = pd.merge(df_ID,taxizones,left_on='PULocationID',right_on='LocationID')

        # Drop irrelevant columns
        df_ID = df_ID[['tpep_pickup_datetime','PULocationID','zone','borough']]

        # Rename columns for merging
        df_ID.rename(columns={'tpep_pickup_datetime':'pickup_date','PULocationID':'LocationID'},
                     inplace=True)

        df_ID['Base'] = 'Yellow'

        # Create Day and Hour Columns
        df_ID.pickup_date = pd.to_datetime(df_ID.pickup_date)
        df_ID['date'] = df_ID.pickup_date.dt.date
        df_ID['hour'] = df_ID.pickup_date.dt.hour
    
    df = pd.concat([df_Loc,df_ID]).reset_index(drop=True)
    return df

In [ ]:
Yellow = ProcessYTData()

Completed 2015-01
Completed 2015-02
Completed 2015-03
Completed 2015-04
Completed 2015-05
Completed 2015-06
Completed 2015-07
Completed 2015-08
Completed 2015-09
Completed 2015-10
Completed 2015-11
Completed 2015-12
Completed 2016-01
Completed 2016-02
Completed 2016-03
Completed 2016-04
Completed 2016-05
Completed 2016-06
Completed 2016-07
Completed 2016-08
Completed 2016-09
Completed 2016-10
Completed 2016-11
Completed 2016-12
Completed 2017-01
Completed 2017-02
Completed 2017-03
Completed 2017-04
Completed 2017-05
Completed 2017-06
Completed 2017-07
Completed 2017-08
Completed 2017-09
Completed 2017-10
Completed 2017-11
Completed 2017-12


In [ ]:
YellowGroup = Yellow.groupby(('date','Base','zone'))['pickup_date'].count().reset_index()
YellowGroup.rename(columns={'pickup_date':'COUNTS'},inplace=True)
YellowGroup.head()

### Green Taxis

In [ ]:
###############################################################################
# Green Taxi Processing Data Function
def GTProcessData():
    """ Need to fill in"""
    # Create Blank Dataframe
    df_Loc = pd.DataFrame(columns=['lpep_pickup_datetime',
                                   'Pickup_longitude',
                                   'Pickup_latitude'])
    df_ID = pd.DataFrame(columns=['lpep_pickup_datetime',
                                  "PULocationID"])
    
    for yr in range(2015,2018):
        # Define year
        year = str(yr)

        for mnth in range(1,13):
            # Define month
            month = ('{:02d}'.format(mnth))
            
            # Download Data and format
            temp = pd.read_csv("Data/green_tripdata_" + 
                               year + "-" + month + ".csv")
            
            if(sum(temp.ix[:,-2:].isnull().sum()/len(temp)) == 2.0):
                #fix indexing problem
                columns = temp.columns
                temp = temp.reset_index()
                temp = temp.drop(columns[-2:], axis=1)
                temp.columns = columns
            
            
            if "Pickup_longitude" in temp.columns:
                temp = temp[['lpep_pickup_datetime',
                             'Pickup_longitude',
                             'Pickup_latitude']]
                df_Loc = pd.concat([df_Loc, temp]).reset_index(drop=True)

            else:
                temp = temp[['lpep_pickup_datetime',"PULocationID"]]
                df_ID = pd.concat([df_ID, temp]).reset_index(drop=True)
            
            
            
            # Provide Status Update
            print("Completed " + str(year) + "-" + str(month))
###############################################################################            
    # Change Name of columns
    if len(df_Loc > 0):
        df_Loc.rename(columns={'lpep_pickup_datetime': 'pickup_date',
                           'Pickup_longitude':'pickup_longitude',
                           'Pickup_latitude':'pickup_latitude'},inplace=True)
        df_Loc['Base'] = 'Green'

##############################################################################

        # Filter out invalid data points (outside of NYC)
        # Drop Coords out of range 
        #West -74.257159 East -73.699215
        #North 40.915568 South 40.495992
        df_Loc = df_Loc[~((df_Loc.pickup_longitude < -74.5) | (df_Loc.pickup_longitude > -73))]
        df_Loc = df_Loc[~((df_Loc.pickup_latitude < 40) | (df_Loc.pickup_latitude > 41))]

##############################################################################

        # Convert Lat Long into point geometry
        crs = {'init':'epsg:4326'}
        geometry = [Point(xy) for xy in zip(df_Loc.pickup_longitude, 
                                            df_Loc.pickup_latitude)]
        df_Loc = GeoDataFrame(df_Loc, crs=crs, geometry=geometry)

        # COnvert Cooords for Spatial join
        df_Loc =  df_Loc.to_crs(epsg=2263)
        print("Coordinates Converted")

        # Conduct Spatial Join to identify tazi zone of pickup
        print("spatial join starting. This may take a while")
        df_Loc = gpd.sjoin(df_Loc, taxizones,op='within')
        print("spatial join complete")

##############################################################################

        # Drop Columns
        df_Loc = df_Loc[['pickup_date','LocationID','zone','borough','Base']]

        # Create a date column
        df_Loc.pickup_date = pd.to_datetime(df_Loc.pickup_date)
        df_Loc['date'] = df_Loc.pickup_date.dt.date
        df_Loc['hour'] = df_Loc.pickup_date.dt.hour
    
############################################################################## 

    if len(df_ID > 0):
        df_ID = pd.merge(df_ID,taxizones,left_on='PULocationID',right_on='LocationID')

        # Drop irrelevant columns
        df_ID = df_ID[['lpep_pickup_datetime','PULocationID','zone','borough']]

        # Rename columns for merging
        df_ID.rename(columns={'lpep_pickup_datetime':'pickup_date','PULocationID':'LocationID'},
                     inplace=True)

        df_ID['Base'] = 'Green'

        # Create Day and Hour Columns
        df_ID.pickup_date = pd.to_datetime(df_ID.pickup_date)
        df_ID['date'] = df_ID.pickup_date.dt.date
        df_ID['hour'] = df_ID.pickup_date.dt.hour

    df = pd.concat([df_Loc,df_ID]).reset_index(drop=True)
    return df

In [ ]:
Green = GTProcessData()

In [ ]:
GreenGroup = Green.groupby(('date','Base','zone'))['pickup_date'].count().reset_index()
GreenGroup.rename(columns={'pickup_date':'COUNTS'},inplace=True)
GreenGroup.head()

### Taxi Base Data

In [ ]:
# Read in Taxi Bases Lists
TLCbasesComm = pd.read_excel("http://www1.nyc.gov/assets/tlc/downloads/datasets/current_community_car_service_bases.xls")
TLCbasesBlack = pd.read_excel("http://www1.nyc.gov/assets/tlc/downloads/datasets/current_black_car_bases.xls")
TLCbasesLux = pd.read_excel("http://www1.nyc.gov/assets/tlc/downloads/datasets/current_luxury_limousine_bases.xls")

In [ ]:
##############################################################################

# Drop Unnecessary Columns and rename ones that are inccorectly labeled
TLCbasesComm = TLCbasesComm[['LICENSEE NUMBER', 'NAME OF LICENSEE', 
                             'ALTERNATE NAME OF LICENSEE']]
TLCbasesBlack = TLCbasesBlack[['LICENSEE NUMBER', 'NAME OF LICENSEE', 
                               'ALTERNATE NAME OF LICENSEE']]
TLCbasesLux = TLCbasesLux[['LICENSEE NUMBER', 'NAME OF LICENSEE', 
                           'ALTERNATIVE NAME OF LICENSEE']]

TLCbasesLux = TLCbasesLux.rename(columns={"ALTERNATIVE NAME OF LICENSEE":
                                          'ALTERNATE NAME OF LICENSEE'})

##############################################################################

# Merge Taxi Bases list and drop irrelevant columns
TLCbases = pd.concat([TLCbasesComm,TLCbasesBlack,TLCbasesLux]).reset_index()
TLCbases.drop(['index'],axis=1,inplace=True)

In [ ]:
TLCbases.tail()

From preliminary analysis and sense check with Uber 538 data we know that we are missing one base for Uber, presumably because the base is no longer used. 

We need to add this base to the list

In [ ]:
# Create data point
tempdf = pd.DataFrame([['B02598',' ','uber' ]], columns=('LICENSEE NUMBER','NAME OF LICENSEE','ALTERNATE NAME OF LICENSEE'))
tempdf

In [ ]:
# Append onto dat TLC bases
TLCbases = TLCbases.append(tempdf)

In [ ]:
# Remove null bases and make all strings lower case
TLCbases.dropna(inplace=True)
TLCbases['ALTERNATE NAME OF LICENSEE'] = TLCbases['ALTERNATE NAME OF LICENSEE'].str.lower()

In [ ]:
# Define all bases that contain uber, lyft and via and others
TLCbases["ALTERNATE NAME OF LICENSEE"][TLCbases["ALTERNATE NAME OF LICENSEE"].str.contains('uber')] = "uber"
TLCbases["ALTERNATE NAME OF LICENSEE"][TLCbases["ALTERNATE NAME OF LICENSEE"].str.contains('lyft')] = "lyft"
TLCbases["ALTERNATE NAME OF LICENSEE"][TLCbases["ALTERNATE NAME OF LICENSEE"].str.contains('via')] = "via"
TLCbases["ALTERNATE NAME OF LICENSEE"][~TLCbases["ALTERNATE NAME OF LICENSEE"].str.contains('uber|lyft|via')] = "other"

In [ ]:
TLCbases.tail()

### FHV data

In [ ]:
###############################################################################
# FHV Taxi Processing Data Function
def ProcessFHVData():
    """ Need to fill in"""
    # Create Blank Dataframe
    df = pd.DataFrame(columns=['Dispatching_base_num', 
                               'Pickup_date', 
                               'locationID'])

##############################################################################

    for yr in range(2015,2018):
        # Define year
        year = str(yr)

        for mnth in range(1,13):
            # Define month
            month = ('{:02d}'.format(mnth))
            
            # Download Data and format
            temp = pd.read_csv("Data/fhv_tripdata_" + 
                               year + "-" + month + ".csv")

            # Fix data format when changed
            if "PUlocationID" in temp.columns:
                # Only select columns that exist in previous data
                temp = temp[["Dispatching_base_num",
                             "Pickup_DateTime","PUlocationID"]]
                # Rename columns for merge
                temp.rename(columns={"PUlocationID":"locationID",
                                     "Pickup_DateTime":"Pickup_date"}, 
                            inplace=True)
            
            # Append to larger dataframe
            df = pd.concat([df, temp]).reset_index(drop=True)
            
            # Provide Status Update
            print("Completed " + str(year) + "-" + str(month))
            
###############################################################################      
    # Data Formatting and merging
    # Merge with Taxi Base data set to determine actual bases
    df = pd.merge(df,TLCbases,left_on='Dispatching_base_num',
                  right_on='LICENSEE NUMBER')

    # Merge Location IDs to get Borough and anme of zone
    df = pd.merge(df,taxizones,left_on='locationID',right_on='LocationID')
    
    # Drop irrelevant columns
    df = df[['Pickup_date','locationID','zone','borough',
             'ALTERNATE NAME OF LICENSEE']]
    
    # Rename columns for merging
    df.rename(columns={'Pickup_date':'pickup_date','locationID':'LocationID',
                       'ALTERNATE NAME OF LICENSEE':'Base'},inplace=True)
    
    # Create Day and Hour Columns
    df.pickup_date = pd.to_datetime(df.pickup_date)
    df['date'] = df.pickup_date.dt.date
    df['hour'] = df.pickup_date.dt.hour

    return df

In [ ]:
FHV = ProcessFHVData()

In [ ]:
FHVgroup = FHV.groupby(('date','Base','zone'))['pickup_date'].count().reset_index()
FHVgroup.rename(columns={'pickup_date':'COUNTS'},inplace=True)
FHVgroup.head()

In [ ]:
pl.plot(FHVgroup['date'],FHVgroup['COUNTS'])

# Merge all taxi data

In [ ]:
# Merge data sets into one
output = pd.concat([FHVgroup,GreenGroup,YellowGroup])
output.head()

In [ ]:
output.to_csv('TaxiGrouped2015-2017.csv')

In [ ]:
YellowGroup2 = Yellow.groupby(('date','Base','zone'))['pickup_date'].count().reset_index()
YellowGroup2.rename(columns={'pickup_date':'COUNTS'},inplace=True)

GreenGroup2 = Green.groupby(('date','hour','Base','zone'))['pickup_date'].count().reset_index()
GreenGroup2.rename(columns={'pickup_date':'COUNTS'},inplace=True)

FHVgroup2 = FHV.groupby(('date','hour','Base','zone'))['pickup_date'].count().reset_index()
FHVgroup2.rename(columns={'pickup_date':'COUNTS'},inplace=True)


output2 = pd.concat([FHVgroup2,GreenGroup2,YellowGroup2])

output.to_csv('TaxiGrouped2015-2017.csv')

In [ ]:
output2.to_csv('TaxiGroupedHour2015-2017.csv')

In [28]:
Yellow.head()

,pickup_date,LocationID,zone,borough,Base,date,hour
0,2015-01-15 19:05:39,186.0,Penn Station/Madison Sq West,Manhattan,Yellow,2015-01-15,19
1,2015-01-10 20:33:41,186.0,Penn Station/Madison Sq West,Manhattan,Yellow,2015-01-10,20
2,2015-01-15 19:05:42,186.0,Penn Station/Madison Sq West,Manhattan,Yellow,2015-01-15,19
3,2015-01-04 13:44:52,186.0,Penn Station/Madison Sq West,Manhattan,Yellow,2015-01-04,13
4,2015-01-15 14:00:45,186.0,Penn Station/Madison Sq West,Manhattan,Yellow,2015-01-15,14
